In [1]:
import math
import pandas as pd 
import numpy as np


data = pd.read_csv('cat1.csv')


# print(data.head())


#features
x = data.drop(['index', 'galex_objid', 'sdss_objid', 'class', 'pred'], axis = 1) 
# print(x.head())


#class
y = data['class']
# print(y.head())

In [2]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))



#fix scientifci notations to normal floats
# def scitofloat(x):
#     return x

In [3]:
#normalise the values of x

outercnt = 0
for index, row in x.iterrows():
    innercnt = 0
    for column in x:
#         x.at[outercnt, column] = scitofloat(x.iloc[outercnt, innercnt])
        x.at[outercnt, column] = sigmoid(x.iloc[outercnt, innercnt])
#         print(outercnt, column, '\t\t', x.iloc[outercnt, innercnt])
        innercnt += 1
    outercnt += 1    

#     print('\n\n')

print(x.head())

     u    g    r    i    z  extinction_u  extinction_g  extinction_r  \
0  1.0  1.0  1.0  1.0  1.0      0.523712      0.518482      0.512789   
1  1.0  1.0  1.0  1.0  1.0      0.526258      0.520468      0.514164   
2  1.0  1.0  1.0  1.0  1.0      0.521345      0.516636      0.511511   
3  1.0  1.0  1.0  1.0  1.0      0.518531      0.514442      0.509992   
4  1.0  1.0  1.0  1.0  1.0      0.512538      0.509770      0.506759   

   extinction_i  extinction_z  ...       g-z       r-i       r-z       i-z  \
0      0.509504      0.507070  ...  0.752323  0.573081  0.617067  0.545544   
1      0.510526      0.507830  ...  0.937413  0.719195  0.797190  0.605481   
2      0.508555      0.506363  ...  0.499443  0.537674  0.501030  0.463350   
3      0.507426      0.505524  ...  0.591679  0.522131  0.574171  0.552384   
4      0.505023      0.503736  ...  0.783204  0.636499  0.716019  0.590153   

    fuv-nuv     fuv-u     fuv-g     fuv-r     fuv-i     fuv-z  
0  0.107239  0.012509  0.003379  0

In [4]:
def info_dataset(data, verbose=True):
    label1, label2 = 0, 0
    data_size = len(data)
    for index, row in data.iterrows():
        label = row['class']
#         print(label)
        if label == 1:
            label1 += 1
        else:
            label2 += 1
    if verbose:
        print('Total of samples: %d' % data_size)
        print('Total label 1: %d' % label1)
        print('Total label 0: %d' % label2)
    return [len(data), label1, label2]

In [5]:
p = 0.6
_, label1, label2 = info_dataset(data,False)

info_dataset(data)

Total of samples: 649
Total label 1: 595
Total label 0: 54


[649, 595, 54]

In [6]:
#get training set and test set

from sklearn.model_selection import train_test_split

xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.2, random_state = 15)

train_set = pd.concat([xTrain, yTrain], axis=1, sort=False)
test_set = pd.concat([xTest, yTest], axis=1, sort=False)


print(train_set.iloc[0:4])
# print(train_set.iloc[0][1])

       u    g    r    i    z  extinction_u  extinction_g  extinction_r  \
425  1.0  1.0  1.0  1.0  1.0      0.514904      0.511614      0.508035   
55   1.0  1.0  1.0  1.0  1.0      0.519491      0.515190      0.510510   
512  1.0  1.0  1.0  1.0  1.0      0.511870      0.509250      0.506399   
487  1.0  1.0  1.0  1.0  1.0      0.532910      0.525657      0.517758   

     extinction_i  extinction_z  ...       r-i       r-z       i-z   fuv-nuv  \
425      0.505971      0.504442  ...  0.491625  0.548138  0.556422  0.299982   
55       0.507811      0.505810  ...  0.522065  0.588425  0.566882  0.361392   
512      0.504755      0.503537  ...  0.494274  0.446529  0.452196  0.418036   
487      0.513198      0.509818  ...  0.501639  0.556020  0.554401  0.272535   

        fuv-u     fuv-g     fuv-r     fuv-i     fuv-z  class  
425  0.306498  0.246320  0.230738  0.236738  0.198245      1  
55   0.152176  0.126272  0.122067  0.112914  0.088632      1  
512  0.060764  0.037318  0.026722  0.02

In [7]:
def euclidian_dist(p1, p2):
    dim, sum_ = len(p1), 0
    for index in range(dim - 1):
        sum_ += math.pow(p1[index] - p2[index], 2)
    return math.sqrt(sum_)

In [8]:
def knn(train_set, new_sample, K):

#     print('train set', train_set, type(train_set))
#     print('new sample', new_sample, type(new_sample))
    dists, train_size = {}, len(train_set)

    for i in range(train_size):
#         print(train_set.iloc[i-1].values.tolist())
#         print(new_sample.values.tolist()[0])
#         print('\n\n')
        d = euclidian_dist(train_set.iloc[i-1].values.tolist(), new_sample.values.tolist()[0])
        dists[i] = d
        
#     print('done')
    
    k_neighbors = sorted(dists, key=dists.get)[:K]
    
    qty_label1, qty_label2 = 0, 0
    
    for index in k_neighbors:
#         print('index, ', index)
#         print(train_set.iloc[index])
        
    
        if train_set.iloc[index].values.tolist()[-1] == 1:
            qty_label1 += 1
        else:
            qty_label2 += 1
        
            
    if qty_label1 > qty_label2:
        return 1
    else:
        return 0

In [10]:
correct, K = 0, 3

count, tp, fp, tn, fn = 0, 0, 0, 0, 0


for index, sample in test_set.iterrows():
    print(count)
    count +=1
    sample = pd.DataFrame(sample)
    sample = sample.T

    label = knn(train_set, sample, K)

    if label == sample.iloc[0].values.tolist()[-1]:
        correct += 1
    
    print('predicted, actual', label, sample.iloc[0].values.tolist()[-1])
    if label == 1 and sample.iloc[0].values.tolist()[-1] == 1:
        tp += 1
    if label == 1 and sample.iloc[0].values.tolist()[-1] == 0:
        fp += 1 
    if label == 0 and sample.iloc[0].values.tolist()[-1] == 1:
        fn += 1    
    if label == 0 and sample.iloc[0].values.tolist()[-1] == 0:
        tn += 1    

print('total: ', count)
print('correct: ', correct)
print('\n\n')

matrix_list = np.array([[tp, fp], [fn, tn]])

# matrix = pd.DataFrame({'Actual +ve': matrix_list[:, 0], 'Actual -ve': matrix_list[:, 1]})
matrix = pd.DataFrame(matrix_list, index=['Pred +ve', 'Pred -ve'], columns=['Actual +ve', 'Actual -ve'])



print(matrix)

0
predicted, actual 1 1.0
1
predicted, actual 0 1.0
2
predicted, actual 1 1.0
3
predicted, actual 1 1.0
4
predicted, actual 1 1.0
5
predicted, actual 1 1.0
6
predicted, actual 1 1.0
7
predicted, actual 1 1.0
8
predicted, actual 1 1.0
9
predicted, actual 1 1.0
10
predicted, actual 1 1.0
11
predicted, actual 1 1.0
12
predicted, actual 1 1.0
13
predicted, actual 1 1.0
14
predicted, actual 1 1.0
15
predicted, actual 1 1.0
16
predicted, actual 1 1.0
17
predicted, actual 1 0.0
18
predicted, actual 1 1.0
19
predicted, actual 1 1.0
20
predicted, actual 1 1.0
21
predicted, actual 1 1.0
22
predicted, actual 1 1.0
23
predicted, actual 1 1.0
24
predicted, actual 1 1.0
25
predicted, actual 1 1.0
26
predicted, actual 1 1.0
27
predicted, actual 1 0.0
28
predicted, actual 1 1.0
29
predicted, actual 1 1.0
30
predicted, actual 1 1.0
31
predicted, actual 1 1.0
32
predicted, actual 1 1.0
33
predicted, actual 1 1.0
34
predicted, actual 1 1.0
35
predicted, actual 1 1.0
36
predicted, actual 1 1.0
37
predicte

In [11]:
print("Train set size: %d" % len(train_set))
print("Test set size: %d" % len(test_set))
print("Correct predicitons: %d" % correct)
print("Accuracy: %.2f%%" % (100 * correct / len(test_set)))

precision = tp/(tp + fp)
recall = tp/(tp + fn)
fscore = 2*((precision * recall)/(precision + recall))

print('precision: ', precision)
print('recall: ', recall)
print('fscore: ', fscore)


Train set size: 519
Test set size: 130
Correct predicitons: 116
Accuracy: 89.23%
precision:  0.9133858267716536
recall:  0.9747899159663865
fscore:  0.943089430894309


In [ ]:
#